# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(42)
initial_params = np.random.random([13])

INITIALIZATION_METHOD = 'Amplitude'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label in [0, 2, 4, 6, 8]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=3)

In [9]:
def unitary(params, wire1, wire2):
    # qml.RZ(0, wires=wire1)
    qml.RY(params[0], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[1], wires=wire2)
    # qml.RZ(0, wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[2], wires=wire2)
    qml.CNOT(wires=[wire1, wire2])
    qml.RY(params[3], wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[4], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[5], wires=wire2)
    # qml.RZ(0, wires=wire2)

In [10]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(3), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(3), rotation='Y')

    # First layer
    unitary(params[0:6], 0, 1)

    # Second layer
    unitary(params[6:12], 1, 2)

    # Third layer
    qml.RY(params[12], wires=2)

    # Measurement
    return qml.expval(qml.PauliZ(2))

## Circuit example

In [11]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264]

Example features: [-1.61705485  0.69044237  0.77197159 -0.25862626  1.33467915  2.79386831
  0.62238858 -0.76406815]

Expectation value: -0.4022283136235306

 0: ──╭QubitStateVector(M0)──RY(0.375)──╭X─────────────╭C─────────────╭X──RY(0.156)───────────────────────────────────────────────────────────────────────┤     
 1: ──├QubitStateVector(M0)──RY(0.951)──╰C──RY(0.732)──╰X──RY(0.599)──╰C──RY(0.156)──RY(0.0581)──╭X─────────────╭C─────────────╭X──RY(0.0206)─────────────┤     
 2: ──╰QubitStateVector(M0)──RY(0.866)───────────────────────────────────────────────────────────╰C──RY(0.601)──╰X──RY(0.708)──╰C──RY(0.97)────RY(0.832)──┤ ⟨Z⟩ 
M0 =
[-0.42742986+0.j  0.18250197+0.j  0.20405227+0.j -0.06836168+0.j
  0.35279059+0.j  0.73849242+0.j  0.16451357+0.j -0.20196318+0.j]



# Accuracy test definition

In [12]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [13]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.451140456182473
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.4787515006002401
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.5066026410564226
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.5793517406962785
best accuracy so far!
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.6108043217286915
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.6108043217286915
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.6470588235294117
best accuracy so far!
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.6427370948379352
epoch 71

epoch 342
epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.692436974789916
test_accuracies: [0.6470588235294117, 0.6427370948379352, 0.6333733493397359, 0.6331332533013205, 0.6436974789915966, 0.6523409363745498, 0.6499399759903961, 0.6535414165666267, 0.6645858343337334, 0.6607442977190876, 0.6614645858343338, 0.6741896758703481, 0.6864345738295319, 0.6744297719087635, 0.6842737094837935, 0.6797118847539015, 0.6821128451380551, 0.6840336134453782, 0.6840336134453782, 0.6823529411764706, 0.6782713085234093, 0.6840336134453782, 0.6739495798319328, 0.6854741896758704, 0.6967587034813926, 0.6799519807923169, 0.6873949579831933, 0.690516206482593, 0.6864345738295319, 0.692436974789916]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.6720288115246098
test_accuracies: [0.6427370948379352, 0.6333733493397359, 0.6331332533013205, 0.6436974789915966, 0.6523409363745498, 0.6499399759903961,

epoch 454
epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.6907563025210084
test_accuracies: [0.6741896758703481, 0.6864345738295319, 0.6744297719087635, 0.6842737094837935, 0.6797118847539015, 0.6821128451380551, 0.6840336134453782, 0.6840336134453782, 0.6823529411764706, 0.6782713085234093, 0.6840336134453782, 0.6739495798319328, 0.6854741896758704, 0.6967587034813926, 0.6799519807923169, 0.6873949579831933, 0.690516206482593, 0.6864345738295319, 0.692436974789916, 0.6720288115246098, 0.6837935174069627, 0.695078031212485, 0.6912364945978391, 0.685234093637455, 0.6672268907563026, 0.6936374549819928, 0.6933973589435775, 0.6895558223289315, 0.6739495798319328, 0.6907563025210084]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.6835534213685475
test_accuracies: [0.6864345738295319, 0.6744297719087635, 0.6842737094837935, 0.6797118847539015, 0.6821128451380551, 0.6840336134453782, 0.6840336134453782, 

epoch 564
epoch 565
epoch 566
epoch 567
epoch 568
epoch 569
epoch 570
accuracy: 0.6885954381752701
test_accuracies: [0.6739495798319328, 0.6854741896758704, 0.6967587034813926, 0.6799519807923169, 0.6873949579831933, 0.690516206482593, 0.6864345738295319, 0.692436974789916, 0.6720288115246098, 0.6837935174069627, 0.695078031212485, 0.6912364945978391, 0.685234093637455, 0.6672268907563026, 0.6936374549819928, 0.6933973589435775, 0.6895558223289315, 0.6739495798319328, 0.6907563025210084, 0.6835534213685475, 0.6962785114045618, 0.6828331332533013, 0.6974789915966386, 0.6888355342136855, 0.6821128451380551, 0.6765906362545018, 0.6948379351740697, 0.6960384153661465, 0.6782713085234093, 0.6885954381752701]
epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.6715486194477791
test_accuracies: [0.6854741896758704, 0.6967587034813926, 0.6799519807923169, 0.6873949579831933, 0.690516206482593, 0.6864345738295319, 0.692436974789916, 0.

epoch 677
epoch 678
epoch 679
epoch 680
accuracy: 0.6828331332533013
test_accuracies: [0.6912364945978391, 0.685234093637455, 0.6672268907563026, 0.6936374549819928, 0.6933973589435775, 0.6895558223289315, 0.6739495798319328, 0.6907563025210084, 0.6835534213685475, 0.6962785114045618, 0.6828331332533013, 0.6974789915966386, 0.6888355342136855, 0.6821128451380551, 0.6765906362545018, 0.6948379351740697, 0.6960384153661465, 0.6782713085234093, 0.6885954381752701, 0.6715486194477791, 0.6869147659063626, 0.6869147659063626, 0.6818727490996399, 0.6854741896758704, 0.690516206482593, 0.6813925570228091, 0.6929171668667466, 0.682593037214886, 0.6888355342136855, 0.6828331332533013]
epoch 681
epoch 682
epoch 683
epoch 684
epoch 685
epoch 686
epoch 687
epoch 688
epoch 689
epoch 690
accuracy: 0.6763505402160864
test_accuracies: [0.685234093637455, 0.6672268907563026, 0.6936374549819928, 0.6933973589435775, 0.6895558223289315, 0.6739495798319328, 0.6907563025210084, 0.6835534213685475, 0.69627851

epoch 788
epoch 789
epoch 790
accuracy: 0.6847539015606243
test_accuracies: [0.6974789915966386, 0.6888355342136855, 0.6821128451380551, 0.6765906362545018, 0.6948379351740697, 0.6960384153661465, 0.6782713085234093, 0.6885954381752701, 0.6715486194477791, 0.6869147659063626, 0.6869147659063626, 0.6818727490996399, 0.6854741896758704, 0.690516206482593, 0.6813925570228091, 0.6929171668667466, 0.682593037214886, 0.6888355342136855, 0.6828331332533013, 0.6763505402160864, 0.6907563025210084, 0.6866746698679471, 0.6794717887154862, 0.6813925570228091, 0.687875150060024, 0.6883553421368547, 0.683313325330132, 0.6821128451380551, 0.6890756302521008, 0.6847539015606243]


In [14]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [-0.28334645 -1.1845177   0.62900134  1.16722326  0.15601864 -0.32253179
 -0.4204427  -1.03298208 -0.35521813  0.70946129  0.02058449  0.89614071
  0.7586735 ]


# Testing

In [15]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.6902857142857143


In [16]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 9.396794557571411
training time: 1691.6299695968628
test time: 47.91067862510681
total time: 1748.937442779541
